In [1]:
from probemapper.registration import ANTSPointTransformation, ANTSTransformationStages
from probemapper.io import save_nifti
import os, subprocess
import numpy as np
import pandas as pd

In [2]:
rootdir = "/bucket/ReiterU/DBS/data/20220321_P177_laqueus_DiI_3-8_CUBIC/"
resultdir = os.path.join(rootdir, "analysis")
if not os.path.exists(resultdir):
    subprocess.run(["ssh", "deigo", "mkdir", resultdir])

In [3]:
cwd = os.path.basename(os.getcwd())
tmpdir = os.path.join("/flash/ReiterU/tmp/", cwd, f"indexing")
if not os.path.exists(tmpdir):
    os.makedirs(tmpdir)

### Transform channel coordinates

In [4]:
T = ANTSPointTransformation(
    ants_path="/apps/unit/ReiterU/ANTs/2.3.5/bin/",
    num_threads=8
)
T.set_transformations([
    ANTSTransformationStages(transformation=os.path.join(rootdir, "analysis/ants/F2M_0GenericAffine.mat"), inverse=True),
    ANTSTransformationStages(transformation=os.path.join(rootdir, "analysis/ants/F2M_1InverseWarp.nii.gz"), inverse=False),
])
T.set_moving_points(os.path.join(rootdir, "analysis/skeleton/channel_skeleton.csv"))
T.set_outname(os.path.join(tmpdir, "channel_LUT.csv"))

In [5]:
T.run()

['/apps/unit/ReiterU/ANTs/2.3.5/bin/antsApplyTransformsToPoints', '-d', '3', '-i', '/bucket/ReiterU/DBS/data/20220321_P177_laqueus_DiI_3-8_CUBIC/analysis/skeleton/channel_skeleton.csv', '-o', '/flash/ReiterU/tmp/P177/indexing/channel_LUT.csv', '-t', '[/bucket/ReiterU/DBS/data/20220321_P177_laqueus_DiI_3-8_CUBIC/analysis/ants/F2M_0GenericAffine.mat,1]', '-t', '/bucket/ReiterU/DBS/data/20220321_P177_laqueus_DiI_3-8_CUBIC/analysis/ants/F2M_1InverseWarp.nii.gz']



### Query region ID

In [6]:
from probemapper import skeletonization

In [7]:
df = pd.read_csv(os.path.join(tmpdir, "channel_LUT.csv"))
coords = np.array([df["Z"], df["Y"], df["X"]])

In [9]:
regions = skeletonization.query_region_id2(coords,
                     "/bucket/ReiterU/DBS/atlas/O_Laqueues_v0.1/Slicer_3D/Segmentation.seg.nrrd",
                     100, coords.shape[1]-100, 10)

In [10]:
df["Region"] = regions

In [11]:
df.insert(0, "channel", df.pop("channel"))

In [12]:
display(df)

,channel,X,Y,Z,X_org,Y_org,Z_org,Region
0,-100,2292.64,2464.960,1289.13,2201.52,2757.720,683.020,3
1,-99,2290.62,2457.180,1294.51,2198.23,2749.450,685.180,3
2,-98,2288.59,2449.410,1299.89,2194.94,2741.180,687.339,3
3,-97,2286.56,2441.670,1305.25,2191.64,2732.910,689.498,3
4,-96,2284.52,2433.950,1310.60,2188.35,2724.640,691.658,3
...,...,...,...,...,...,...,...,...
336,236,2159.05,1009.520,3542.41,1267.05,403.548,2494.910,1
337,237,2164.00,1001.310,3551.37,1265.76,395.258,2501.040,1
338,238,2168.64,992.690,3560.34,1264.15,386.941,2506.860,1
339,239,2172.82,983.633,3569.19,1262.19,378.597,2512.360,1


In [13]:
df.to_csv(os.path.join(tmpdir, "channel_LUT.csv"), index=False)

In [14]:
sk = skeletonization.generate_skeleton_image(coords[:, 100:],
                             "/bucket/ReiterU/DBS/atlas/O_Laqueues_v0.1/Slicer_3D/Segmentation.seg.nrrd",
                             10)

In [15]:
sk = sk.astype("uint8")
save_nifti(sk, os.path.join(tmpdir, "probe_image.nii.gz"), 1, 1, 1)

In [16]:
subprocess.run(["scp", "-r", tmpdir, f"deigo:{resultdir}"])

CompletedProcess(args=['scp', '-r', '/flash/ReiterU/tmp/P177/indexing', 'deigo:/bucket/ReiterU/DBS/data/20220321_P177_laqueus_DiI_3-8_CUBIC/analysis'], returncode=0)